In [1]:
import pandas as pd
import numpy as np

### Stock Data

In [2]:
s = pd.read_excel(r"C:\Users\SS\Documents\Internship - Stock Prediction\Data\IBM.xlsx")

In [3]:
s.head()

,Datetime,Open,High,Low,Close,Volume,Adj Open
0,2020-06-08 09:30:00-04:00,132.270004,132.949997,132.270004,132.949997,145398,133.009995
1,2020-06-08 09:31:00-04:00,133.009995,133.169998,132.949997,133.020004,27135,133.089996
2,2020-06-08 09:32:00-04:00,133.089996,133.520004,133.089996,133.520004,12053,133.529999
3,2020-06-08 09:33:00-04:00,133.529999,133.690002,133.410004,133.639999,17842,133.610001
4,2020-06-08 09:34:00-04:00,133.610001,133.610001,133.500397,133.514999,22657,133.550003


In [4]:
s.columns=["Date","Open","High","Low","Close","Volume","Adj Open"]

In [5]:
s.Date = s.Date.apply(np.datetime64)

In [6]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8129 entries, 0 to 8128
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      8129 non-null   datetime64[ns]
 1   Open      8129 non-null   float64       
 2   High      8129 non-null   float64       
 3   Low       8129 non-null   float64       
 4   Close     8129 non-null   float64       
 5   Volume    8129 non-null   int64         
 6   Adj Open  8129 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 444.7 KB


In [7]:
s = s.sort_values(by='Date')

In [8]:
s.reset_index(inplace=True)

In [9]:
print(f"Starting date and time of the Data is {s.Date.min()}  and Last date and time is {s.Date.max()}")

Starting date and time of the Data is 2020-06-08 13:30:00  and Last date and time is 2020-07-09 16:55:12


In [10]:
s = s.iloc[:,1:]
s.head(2)

,Date,Open,High,Low,Close,Volume,Adj Open
0,2020-06-08 13:30:00,132.270004,132.949997,132.270004,132.949997,145398,133.009995
1,2020-06-08 13:31:00,133.009995,133.169998,132.949997,133.020004,27135,133.089996


### Tweets

In [11]:
t = pd.read_csv(r"C:\Users\SS\Documents\Internship - Stock Prediction\Data\tweet IBM.csv")

In [12]:
t.head()

,Unnamed: 0,date,tweet_text
0,0,2020-06-09 23:59:56+00:00,All their regions are down!
1,1,2020-06-09 23:59:53+00:00,Unfortunately no ETA yet. We are waiting on IB...
2,2,2020-06-09 23:59:46+00:00,Would be cool to see if they can manage to imp...
3,3,2020-06-09 23:59:45+00:00,IBM is canceling its facial recognition programs
4,4,2020-06-09 23:59:39+00:00,[Status] [Mel251] Information: IBM Cloud Netwo...


In [13]:
t = t.iloc[:,1:]

In [14]:
t.columns = ["Date","Tweets"]
t.Date = t.Date.apply(np.datetime64)
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62329 entries, 0 to 62328
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    62329 non-null  datetime64[ns]
 1   Tweets  62329 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 974.0+ KB


In [15]:
# Tweets are arranged
t = t.sort_values(by='Date')


### function to join multiple row
join_rows = lambda x: " ".join(x) 

t = t.set_index("Date")

# First Tweets are resampled minutewise then aggregated/joined by function
t = t.resample('T').agg({'Tweets': join_rows})
t.reset_index(inplace=True)

In [16]:
t.head(2)

,Date,Tweets
0,2020-06-09 09:51:00,"Top story: IBM will no longer offer, develop, ..."
1,2020-06-09 09:52:00,Indeed a bold move. IBM chose ethics over mone...


In [17]:
print(f"Starting date and time of the Data is {t.Date.min()}  and Last date and time is {t.Date.max()}")

Starting date and time of the Data is 2020-06-09 09:51:00  and Last date and time is 2020-07-09 23:59:00


In [18]:
#size of tweet dataset after merging data to minute wise
t.shape

(44049, 2)

### Cropping Stock Data

Stock data's date is earlier than tweets data's date

In [19]:
#Stock data starts from 8-6. So Cropping it to make it start from 9-6
s = s.loc[s['Date']>=t['Date'].min()]

In [20]:
print(f"Starting date and time of the Data is {s.Date.min()}  and Last date and time is {s.Date.max()}")

Starting date and time of the Data is 2020-06-09 13:30:00  and Last date and time is 2020-07-09 16:55:12


In [21]:
s.reset_index(inplace=True)
s = s.iloc[:,1:]
s.head(2)

,Date,Open,High,Low,Close,Volume,Adj Open
0,2020-06-09 13:30:00,133.520004,133.600006,133.080002,133.264999,175324,133.274994
1,2020-06-09 13:31:00,133.274994,133.279999,133.080002,133.160004,22922,133.289993


In [22]:
# After Removing Duplicate Dates
s.drop_duplicates(subset ="Date",keep = False, inplace = True)   

In [23]:
# Final Size of Stock Dataset
s.shape

(7735, 7)

### Tweets Data Cleaning according to stock data

In [24]:
# cropping tweet data to match startinf time with stock data
for i in range(len(t)):
    if t.Date.iloc[i]==s.Date.iloc[0]:
        a = i
        break

In [25]:
t = t.iloc[a:,:]
t.reset_index(inplace=True)
t = t.iloc[:,1:]
t.head(2)

,Date,Tweets
0,2020-06-09 13:30:00,[music theory coursera] IBM Microservices WOW...
1,2020-06-09 13:31:00,IBM quits facial recognition and urges debate ...


In [26]:
t.set_index('Date',inplace=True)

In [27]:
# extracting hour and minute from given time
hours = t.index.hour
minutes = t.index.minute

In [28]:
hours = pd.DataFrame(hours, index=t.index)
minute = pd.DataFrame(minutes, index=t.index)
hours.columns = ['hour']
minute.columns = ['minute']

In [29]:
hours.head(2)

,hour
Date,
2020-06-09 13:30:00,13
2020-06-09 13:31:00,13


In [30]:
minute.head(2)

,minute
Date,
2020-06-09 13:30:00,30
2020-06-09 13:31:00,31


In [31]:
t = pd.concat([t, hours,minute], axis = 1)
t.reset_index(inplace=True)

In [32]:
t.head(2)

,Date,Tweets,hour,minute
0,2020-06-09 13:30:00,[music theory coursera] IBM Microservices WOW...,13,30
1,2020-06-09 13:31:00,IBM quits facial recognition and urges debate ...,13,31


In [33]:
# Combining tweets from 20:00:00 hr to next day 12:59:000 hr

In [34]:
# hours for which we dont have stock data
z = [0,1,2,3,4,5,6,7,8,9,10,11,12,20,21,22,23]

In [35]:
for i in range(len(t)):
    if t.hour[i] == 19 and t.minute[i]==59:
        b = i
        pass
    elif t.hour[i] in z:
        t.Tweets[b] = t.Tweets[b] + " " + t.Tweets[i]

C:\Users\SS\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
# Dropping all tweets which are joined in previous day
for i in range(len(t)):
    if t.hour[i] in z:
        t.drop(i,inplace=True)

In [37]:
t.reset_index(inplace=True)
t = t.iloc[:,1:]

In [38]:
# Shape after 1st Operation
t.shape

(12990, 4)

In [39]:
t.head(2)

,Date,Tweets,hour,minute
0,2020-06-09 13:30:00,[music theory coursera] IBM Microservices WOW...,13,30
1,2020-06-09 13:31:00,IBM quits facial recognition and urges debate ...,13,31


In [40]:
#To get time of weekends
x = s.copy()
x.set_index('Date',inplace=True)

In [41]:
hr = x.index.hour
mn = x.index.minute

hr = pd.DataFrame(hr, index=x.index)
mn = pd.DataFrame(mn, index=x.index)
hr.columns = ['hour']
mn.columns = ['minute']

x = pd.concat([x,hr,mn],axis=1)
x.reset_index(inplace=True)

In [42]:
indx_of_time_of_lst_week_day = []
time_of_lst_week_day = []
for i in range(len(x)):
    if x.Date[i].weekday()==0 and x.hour[i] == 13 and x.minute[i] == 30 :
        indx_of_time_of_lst_week_day.append(i-1)
        time_of_lst_week_day.append((x.Date[i-1],x.Date[i]))

In [43]:
indx_of_time_of_lst_week_day

[1452, 3308, 5142, 6678]

In [44]:
time_of_lst_week_day

[(Timestamp('2020-06-12 18:29:00'), Timestamp('2020-06-15 13:30:00')),
 (Timestamp('2020-06-19 18:29:00'), Timestamp('2020-06-22 13:30:00')),
 (Timestamp('2020-06-26 18:29:00'), Timestamp('2020-06-29 13:30:00')),
 (Timestamp('2020-07-02 19:59:00'), Timestamp('2020-07-06 13:30:00'))]

In [45]:
# Joining Weekend Tweets
for i in range(len(t)):
    if t.Date[i] == time_of_lst_week_day[0][0]:
        f = i
    elif t.Date[i]>time_of_lst_week_day[0][0] and t.Date[i]<time_of_lst_week_day[0][1]:
        t.Tweets[f] = t.Tweets[f] + " " + t.Tweets[i]
        
    elif t.Date[i] == time_of_lst_week_day[1][0]:
        c = i
    elif t.Date[i]>time_of_lst_week_day[1][0] and t.Date[i]<time_of_lst_week_day[1][1]:
        t.Tweets[c] = t.Tweets[c] + " " + t.Tweets[i]
        
    elif t.Date[i] == time_of_lst_week_day[2][0]:
        d = i
    elif t.Date[i]>time_of_lst_week_day[2][0] and t.Date[i]<time_of_lst_week_day[2][1]:
        t.Tweets[d] = t.Tweets[d] + " " + t.Tweets[i]
        
    elif t.Date[i] == time_of_lst_week_day[3][0]:
        e = i
    elif t.Date[i]>time_of_lst_week_day[3][0] and t.Date[i]<time_of_lst_week_day[3][1]:
        t.Tweets[e] = t.Tweets[e] + " " + t.Tweets[i]

C:\Users\SS\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\SS\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\SS\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\SS\anaconda3

In [46]:
# Dropping Week ends tweets after joining them
for i in range(len(t)):
    if t.Date[i]>time_of_lst_week_day[0][0] and t.Date[i]<time_of_lst_week_day[0][1]:
        t.drop(i,inplace=True)
        
    elif t.Date[i]>time_of_lst_week_day[1][0] and t.Date[i]<time_of_lst_week_day[1][1]:
        t.drop(i,inplace=True)
        
    elif t.Date[i]>time_of_lst_week_day[2][0] and t.Date[i]<time_of_lst_week_day[2][1]:
        t.drop(i,inplace=True)
        
    elif t.Date[i]>time_of_lst_week_day[3][0] and t.Date[i]<time_of_lst_week_day[3][1]:
        t.drop(i,inplace=True)

In [47]:
t.reset_index(inplace=True)
t = t.iloc[:,1:]

In [48]:
t.shape

(8820, 4)

In [49]:
# Joining Tweets from 13:00:00 hr to 13:29:00 hr
x1 = list(range(30))

In [50]:
for i in range(len(t)):
    if i<8819:
        if t.hour[i]==13 and t.minute[i+1]==0:
            a1 = i
        elif t.hour[i]==13 and t.minute[i] in x1:
            t.Tweets[a1] = t.Tweets[a1]+" "+t.Tweets[i]

C:\Users\SS\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
# Dropping tweets from 13:00 to 13:29 after joining them
for i in range(len(t)):
    if t.hour[i]==13 and t.minute[i] in x1:
        t.drop(i,inplace=True)

In [52]:
t.reset_index(inplace=True)
t = t.iloc[:,1:]

In [53]:
t.shape

(8310, 4)

In [54]:
# date time stamp which are in Tweet data but not in stock data 
dt_of_stk = []
idx_nt_in_stk = []
idx_in_stk = []

for i in range(len(s)):
    dt_of_stk.append(s.Date.iloc[i])
    

for i in range(len(t)):
    if t.Date.iloc[i] not in dt_of_stk:
        idx_nt_in_stk.append(i)
    
    
for i in range(len(t)):
    if t.Date.iloc[i] in dt_of_stk:
        idx_in_stk.append(i)

In [55]:
print(f"length of t = {len(t)}")
print(f"length of s = {len(s)}")
print(f"lenth of index not in stocks but in tweets = {len(idx_nt_in_stk)}")
print(f"legth of index in stock also in tweets = {len(idx_in_stk)}")

length of t = 8310
length of s = 7735
lenth of index not in stocks but in tweets = 575
legth of index in stock also in tweets = 7735


In [56]:
# idx_nt_in_stk

In [57]:
from itertools import groupby
from operator import itemgetter

In [58]:
# Index of timestamp which ar not in stock but are in tweets. Some of them are in continuation
# Code to make a list of continuous indices
z1 = []
for k, g in groupby(enumerate(idx_nt_in_stk), lambda y=(i,x): y[0]-y[1]):
    z1.append(list(map(itemgetter(1), g)))

In [59]:
# Now joing those Tweets
for i in range(len(z1)):
    for j in z1[i]:
        t.Tweets[z1[i][0]-1] = t.Tweets[z1[i][0]-1]+" "+t.Tweets[j]

C:\Users\SS\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [60]:
for i in range(len(z1)):
    for j in z1[i]:
        t.drop(j,inplace=True)

In [61]:
t.reset_index(inplace=True)
t = t.iloc[:,1:]

In [62]:
# Final Tweets Dataset
t.shape

(7735, 4)

# Now Both Data are Structured

### Model Dataset Preparation /  Feature Extraction

In [63]:
stk = s

In [64]:
stk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7735 entries, 0 to 7734
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      7735 non-null   datetime64[ns]
 1   Open      7735 non-null   float64       
 2   High      7735 non-null   float64       
 3   Low       7735 non-null   float64       
 4   Close     7735 non-null   float64       
 5   Volume    7735 non-null   int64         
 6   Adj Open  7735 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 483.4 KB


In [65]:
stk['Target'] = stk['Open'].shift(-1)
stk.dropna(inplace=True)

In [66]:
stk.set_index("Date",inplace=True)
stk.head(2)

,Open,High,Low,Close,Volume,Adj Open,Target
Date,,,,,,,
2020-06-09 13:30:00,133.520004,133.600006,133.080002,133.264999,175324,133.274994,133.274994
2020-06-09 13:31:00,133.274994,133.279999,133.080002,133.160004,22922,133.289993,133.289993


In [67]:
target = stk.iloc[:,-1]
stk = stk.iloc[:,:-1]

In [68]:
stk.describe()

,Open,High,Low,Close,Volume,Adj Open
count,7734.000000,7734.000000,7734.000000,7734.000000,7.734000e+03,7734.000000
mean,121.580677,121.637752,121.520068,121.576913,1.147265e+04,121.576468
std,4.132462,4.133789,4.129018,4.130738,2.671715e+04,4.132128
min,115.980003,116.069901,115.879997,115.970001,0.000000e+00,115.980003
25%,118.830002,118.879997,118.779999,118.820000,4.600500e+03,118.824997
50%,120.279999,120.330002,120.209999,120.279999,7.231000e+03,120.279999
75%,123.610001,123.653299,123.559998,123.610001,1.254125e+04,123.610001
max,133.520004,133.600006,133.080002,133.264999,1.682168e+06,133.289993


In [69]:
print(f"Starting Data of the Data is {stk.index.min()}  and Last Date is {stk.index.max()}")

Starting Data of the Data is 2020-06-09 13:30:00  and Last Date is 2020-07-08 18:28:00


In [70]:
txt = t.iloc[:7734,:2]

In [71]:
txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7734 entries, 0 to 7733
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    7734 non-null   datetime64[ns]
 1   Tweets  7734 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 121.0+ KB


In [72]:
txt.head()

,Date,Tweets
0,2020-06-09 13:30:00,[music theory coursera] IBM Microservices WOW...
1,2020-06-09 13:31:00,IBM quits facial recognition and urges debate ...
2,2020-06-09 13:32:00,IBM Quits Facial Recognition Over Rights Conce...
3,2020-06-09 13:33:00,"IBM gets out of #facialrecognition business, c..."
4,2020-06-09 13:34:00,IBM ends all facial recognition business as CE...


In [73]:
txt.iloc[:,1:].describe()

,Tweets
count,7734
unique,6228
top,
freq,1450


In [74]:
print(f"Starting Data of the Data is {txt.Date.min()}  and Last Date is {txt.Date.max()}")

Starting Data of the Data is 2020-06-09 13:30:00  and Last Date is 2020-07-08 18:28:00


In [75]:
# For Textual Data Preprocessing
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [76]:
# Cleaning Tweets to calculate Subjactivity and Polarity & later on to calculate more things
'''
1. to remove multiple #hastags written next to one another
2. Same as 1
3. to remove text between () and []
4. to remove URLs
5. to remove some punctuations like @,#,\,;
6. to remove text like via @nvjkxz
7. to remove text like pic.nsdkjacsk
8. to replace multiple space with single space'''


def clean_fun(text):
    text = re.sub("[\(\[].*?[\)\]]", "", str(text)) 
    text = re.sub("\w+:\/\/\S+","",text) 
    text = re.sub("[\@\#\\\;]", " ", text)
    text = re.sub("via @.*?\s+","",text)
    text = re.sub("pic.*?\s+","",text)
    text = re.sub("[\']+","",text)
    text = re.sub(r"\s+"," ",text,flags=re.I)
    return text

In [77]:
txt.Tweets = txt.Tweets.apply(clean_fun)

**Polarity and Subjectivity**

In [78]:
from textblob import TextBlob

In [79]:
twt = txt.copy()
twt['Polarity']=twt.apply(lambda x:TextBlob(x['Tweets']).sentiment.polarity,axis=1)
twt['Subjectivity']=twt.apply(lambda x:TextBlob(x['Tweets']).sentiment.subjectivity,axis=1)

In [80]:
pol_sub = twt[['Polarity','Subjectivity']]
pol_sub.set_index(stk.index,inplace=True)

In [81]:
pol_sub.head()

,Polarity,Subjectivity
Date,,
2020-06-09 13:30:00,0.099753,0.391667
2020-06-09 13:31:00,0.240000,0.240000
2020-06-09 13:32:00,-0.015625,0.046875
2020-06-09 13:33:00,0.004167,0.270833
2020-06-09 13:34:00,0.106612,0.200826


**Named Entity Recognition**

In [82]:
# we use Named Entity Recognition to find number of "PERSON" , "ORGANIZATION" and "LOCATION" in tweets.
from nltk import word_tokenize, ne_chunk, pos_tag
from nltk import tree2conlltags

In [83]:
"""def NER_FUNCTION(text):
    NER = pd.DataFrame(columns=["B-PERSON","I-PERSON","B-ORGANIZATION","I-ORGANIZATION","B-LOCATION","I-LOCATION","B-GPE","I-GPE"])
    ne_tree = ne_chunk(pos_tag(word_tokenize(text)))
    iob_tagged = tree2conlltags(ne_tree)
    Df = pd.DataFrame(iob_tagged)
    Df.columns = ['words', 'POS', 'ENTITY']
    Df = Df[['words', 'ENTITY']]
    group = Df.groupby(['ENTITY'])
    NERForSent1 = (group.agg(np.count_nonzero))
    a = NER.columns[~(NER.columns.isin(NERForSent1.T.columns))]
    NER1 = NER[a]
    NER1.iloc[:,:] = np.repeat(0,len(a))
    NER1 = pd.DataFrame(np.repeat(0,len(a)),NER.columns[~(NER.columns.isin(NERForSent1.T.columns))])
    NER1 = NER1.T
    NERForSent1 = NERForSent1.T
    NERForSent1.index = NER1.index
    NER1 = pd.concat([NER1,NERForSent1],axis=1)
    #del NER1['O']
    NER1 = NER1[["B-PERSON","I-PERSON","B-ORGANIZATION","I-ORGANIZATION","B-LOCATION","I-LOCATION","B-GPE","I-GPE"]]
    return NER1.iloc[0,:]"""

'def NER_FUNCTION(text):\n    NER = pd.DataFrame(columns=["B-PERSON","I-PERSON","B-ORGANIZATION","I-ORGANIZATION","B-LOCATION","I-LOCATION","B-GPE","I-GPE"])\n    ne_tree = ne_chunk(pos_tag(word_tokenize(text)))\n    iob_tagged = tree2conlltags(ne_tree)\n    Df = pd.DataFrame(iob_tagged)\n    Df.columns = [\'words\', \'POS\', \'ENTITY\']\n    Df = Df[[\'words\', \'ENTITY\']]\n    group = Df.groupby([\'ENTITY\'])\n    NERForSent1 = (group.agg(np.count_nonzero))\n    a = NER.columns[~(NER.columns.isin(NERForSent1.T.columns))]\n    NER1 = NER[a]\n    NER1.iloc[:,:] = np.repeat(0,len(a))\n    NER1 = pd.DataFrame(np.repeat(0,len(a)),NER.columns[~(NER.columns.isin(NERForSent1.T.columns))])\n    NER1 = NER1.T\n    NERForSent1 = NERForSent1.T\n    NERForSent1.index = NER1.index\n    NER1 = pd.concat([NER1,NERForSent1],axis=1)\n    #del NER1[\'O\']\n    NER1 = NER1[["B-PERSON","I-PERSON","B-ORGANIZATION","I-ORGANIZATION","B-LOCATION","I-LOCATION","B-GPE","I-GPE"]]\n    return NER1.iloc[0,:]'

In [84]:
"""temp = txt['Tweets'].apply(lambda x: NER_FUNCTION(x))

NER = pd.DataFrame(columns = ["PERSON","ORGANIZATION","LOCATION"]) # Empty Dataframe with 3 columns
NER['PERSON'] = temp[['B-PERSON',"I-PERSON"]].sum(axis=1)
NER['ORGANIZATION'] = temp[['B-ORGANIZATION',"I-ORGANIZATION"]].sum(axis=1)
NER['LOCATION'] = temp[['B-LOCATION',"I-LOCATION",'B-GPE',"I-GPE"]].sum(axis=1)"""

'temp = txt[\'Tweets\'].apply(lambda x: NER_FUNCTION(x))\n\nNER = pd.DataFrame(columns = ["PERSON","ORGANIZATION","LOCATION"]) # Empty Dataframe with 3 columns\nNER[\'PERSON\'] = temp[[\'B-PERSON\',"I-PERSON"]].sum(axis=1)\nNER[\'ORGANIZATION\'] = temp[[\'B-ORGANIZATION\',"I-ORGANIZATION"]].sum(axis=1)\nNER[\'LOCATION\'] = temp[[\'B-LOCATION\',"I-LOCATION",\'B-GPE\',"I-GPE"]].sum(axis=1)'

In [85]:
# NER.set_index(stk.index,inplace=True)
# NER

**TF-IDF Vectorizor**

In [86]:
#Creating Object ofLemmatizer
wordnet = WordNetLemmatizer()

In [87]:
tweetz = []
for i in range(len(twt)):
    tweetz.append(twt.Tweets[i])

In [88]:
'''
1. All characters other than alphabates are removed.
2. All characters are lowered to lowercase.
3. Each word of the headline are splited.
4. Word other than stop words are lemmatizied.
5. Each lemmatized word are finally joined and appended in new list.
'''

tweets = []
for i in range(len(tweetz)):
    text = re.sub('[^a-zA-Z]', ' ', tweetz[i])  
    text = re.sub("via","",text)
    text = text.lower()
    text = text.split()
    text = set(wordnet.lemmatize(word) for word in text if not word in set(stopwords.words('english')))
    text = ' '.join(text)
    tweets.append(text)

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [90]:
TFIDF_obj = TfidfVectorizer(max_features=100)
tweet_array = TFIDF_obj.fit_transform(tweets).toarray()
TFIDF_twt_features = pd.DataFrame(tweet_array, index = stk.index, columns=TFIDF_obj.get_feature_names())

In [91]:
TFIDF_twt_features.head()

,ai,also,amazon,amp,amzn,apple,back,big,blockchain,business,...,virtual,want,watson,way,webinar,work,working,world,would,year
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-09 13:30:00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.189439,...,0.000000,0.0,0.0,0.0,0.0,0.215774,0.000000,0.0,0.000000,0.0
2020-06-09 13:31:00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.305367,0.0,0.000000,0.0
2020-06-09 13:32:00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.188622,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.257758,0.0
2020-06-09 13:33:00,0.185695,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.192321,...,0.263573,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.262812,0.0
2020-06-09 13:34:00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.270093,0.0,0.205431,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


# Model Dataset

In [92]:
Model_data = pd.concat([stk,pol_sub,TFIDF_twt_features,target],axis=1)

In [93]:
Model_data.head(2)

,Open,High,Low,Close,Volume,Adj Open,Polarity,Subjectivity,ai,also,...,want,watson,way,webinar,work,working,world,would,year,Target
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-09 13:30:00,133.520004,133.600006,133.080002,133.264999,175324,133.274994,0.099753,0.391667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.215774,0.000000,0.0,0.0,0.0,133.274994
2020-06-09 13:31:00,133.274994,133.279999,133.080002,133.160004,22922,133.289993,0.240000,0.240000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.305367,0.0,0.0,0.0,133.289993


In [94]:
Model_data.tail(2)

,Open,High,Low,Close,Volume,Adj Open,Polarity,Subjectivity,ai,also,...,want,watson,way,webinar,work,working,world,would,year,Target
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-08 18:27:00,117.220001,117.221802,117.169998,117.199997,5216,117.209999,0.103704,0.42963,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.339156,117.209999
2020-07-08 18:28:00,117.209999,117.254997,117.209999,117.229797,9265,117.230003,0.000000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,117.230003
